In [15]:
!pip install ipython-autotime
%load_ext autotime

Defaulting to user installation because normal site-packages is not writeable
Could not build wheels for ipython-autotime, since package 'wheel' is not installed.
    Running setup.py install for ipython-autotime ... done
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.


In [16]:
cd = {1:4, 2:6, 3:1, 4:8, 5:3, 10:2}

time: 628 µs


In [ ]:
list = []
for key in cd.keys():
    for i in range(cd[key]):
        list.append(key)
list = sorted(list)

In [ ]:
cd = list

In [ ]:
cd

In [ ]:
def quick_median(count_dict: dict) -> int:
    """
    Finds the median of a counting dictionary
    :param count_dict: the counting dictionary to find the median of
    :return: integer index of the location of the median
    """
    list = []
    for key in count_dict.keys():
        for i in range(count_dict[key]):
            list.append(key)
    list = sorted(list)
    midpoint = len(list)//2
    if len(list)%2 == 0:
        median = (list[midpoint] + list[midpoint-1])//2
    else:
        median = list[midpoint]
    return median

In [ ]:
def quick_median_old(count_dict: dict) -> int:
    mid_point = sum(count_dict.values()) / 2
    my_sum = 0
    my_ind = 0
    sk = sorted(count_dict.keys())
    while my_sum < mid_point:
        my_sum += count_dict[sk[my_ind]]
        if my_sum >= mid_point:
            break
        my_ind += 1
    return sk[my_ind]

In [ ]:
cd = {4: 10, 236: 13, 99: 43, 88: 43, 127: 15}

In [ ]:
quick_median(cd)

In [ ]:
quick_median_old(cd)

In [ ]:
cd2 = {4: 10, 236: 13, 99: 43, 88: 43, 127: 14, 8:102}

In [ ]:
quick_median(cd2)

In [ ]:
quick_median_old(cd2)

In [ ]:
my_median1 = quick_median(cd)
my_median2 = quick_median(cd2)

In [ ]:
def median_deviation_from_median1(count_dict: dict) -> int:
    """
    calculates the deviation from the median of each element of counting dictionary,
    then returns the median of that dictionary
    :param count_dict: Counting dictionary to analyze
    :return: index of median of the deviations
    """
    my_median = quick_median1(count_dict)
    deviations = {}
    for key in sorted(count_dict.keys()):
        X_value = abs(key - my_median)
        deviations[X_value] = count_dict[key]
    print(deviations)
    return quick_median1(deviations)

In [ ]:
median_deviation_from_median(cd)

In [ ]:
print(sorted([abs(x-99) for x in list]))

In [ ]:
cd3 = {1:100, 100:100}

In [ ]:
quick_median(cd3)

In [ ]:
quick_median_old(cd3)

In [ ]:
def quick_median1(count_dict: dict) -> int:
    mid_point = sum(count_dict.values()) / 2
    my_sum = 0
    my_ind = 0
    sk = sorted(count_dict.keys())
    while my_sum < mid_point:
        my_sum += count_dict[sk[my_ind]]
        if my_sum >= mid_point:
            break
        my_ind += 1
    if sum(count_dict.values())%2 == 0:
        median = (sk[my_ind] + sk[my_ind-1])//2
    else:
        median = sk[my_ind]
    return median

In [ ]:
quick_median_new(cd)

In [ ]:
def count_frags1(file: str) -> dict:
    """
    Takes a sam file input and creates a counting dictionary of the number of reads that are paired,
    first in the pair, confidently mapped and whose pair is mapped to the same reference
    :param file: A sam input file
    :return: A dictionary of the counts of the above reads
    """
    FILTER_MAPQUAL = 10  # only consider reads that are mapped with at least this mapping quality
    count_dict = {}
    PRINT_EVERY = 100000
    # Check if the file is sam or bam and decide how to open based on that
    if file[-4:] == ".sam":
        file_to_parse = open(file, 'r')
    elif file[-4:] == ".bam":
        file_to_parse = pysam.AlignmentFile(file, 'rb')
    else:
        print("Unknown file type, must be bam or sam")
        exit(1)

    for item in file_to_parse:
        # Need to convert bam iterable objects into strings for the next part
        line = str(item)
        # Skip all comments and headers
        if line[0] == '#' or line[0] == '@':
            continue
        splt = line.strip().split('\t')
        samFlag = int(splt[1])
        myRef = splt[2]
        mapQual = int(splt[4])
        mateRef = splt[6]
        myTlen = abs(int(splt[8]))

        # if read is paired, and is first in pair, and is confidently mapped...
        if samFlag & 1 and samFlag & 64 and mapQual > FILTER_MAPQUAL:
            # and mate is mapped to same reference
            if mateRef == '=' or mateRef == myRef:
                if myTlen not in count_dict:
                    count_dict[myTlen] = 0
                count_dict[myTlen] += 1
    return count_dict

In [ ]:
cd = count_frags("C:/Users/joshf/Documents/work_stuff/neat_data/baby.sam")

In [ ]:
FILTER_MAPQUAL = 10  # only consider reads that are mapped with at least this mapping quality
count_dict = {}
PRINT_EVERY = 100000
i = 0

In [ ]:
for item in file_to_parse:
    # Need to convert bam iterable objects into strings for the next part
    line = str(item)
    # Skip all comments and headers
    if line[0] == '#' or line[0] == '@':
        continue
    splt = line.strip().split('\t')
    samFlag = int(splt[1])
    myRef = splt[2]
    mapQual = int(splt[4])
    mateRef = splt[6]
    myTlen = abs(int(splt[8]))

In [ ]:
len(count_dict)

In [ ]:
def compute_probs1(count_dict: dict) -> (list, list):
    """
    Computes the probabilities for fragments with at least 100 pairs supporting it and that are at least 10 median
    deviations from the median.
    :param count_dict: A dictionary of fragments with counts
    :return: A list of values that meet the criteria and a list of -their associated probabilities
    """
    FILTER_MINREADS = 100  # only consider fragment lengths that have at least this many read pairs supporting it
    FILTER_MEDDEV_M = 10  # only consider fragment lengths this many median deviations above the median
    values = []
    probabilities = []
    med = quick_median1(count_dict)
    mdm = median_deviation_from_median1(count_dict)

    for key in sorted(count_dict.keys()):
        if 0 < key < med + FILTER_MEDDEV_M * mdm:
            if count_dict[key] >= FILTER_MINREADS:
                print(key, count_dict[key])
                values.append(key)
                probabilities.append(count_dict[key])
    count_sum = float(sum(probabilities))
    probabilities = [n / count_sum for n in probabilities]
    return values, probabilities

In [ ]:
compute_probs(cd2)

In [ ]:
import timeit

In [ ]:
def wrapper(func, *args, **kwargs):
    def wrapped():
        return func(*args, **kwargs)
    return wrapped

In [7]:
import pysam

In [ ]:
cd = count_frags("/home/joshfactorial/Documents/neat_data/WGS_chr20_21_22_normal.bam")

In [ ]:
wrapped = wrapper(quick_median, cd)
timeit.timeit(wrapped, number=100)

In [ ]:
wrapped2 = wrapper(quick_median_new, cd)
timeit.timeit(wrapped2, number=100)

In [25]:
def quick_median_new2(count_dict: dict) -> int:
    mid_point = sum(count_dict.values()) / 2
    print(mid_point)
    my_sum = 0
    my_ind = 0
    sk = sorted(count_dict.keys())
    while my_sum < mid_point:
        my_sum += count_dict[sk[my_ind]]
        if my_sum >= mid_point:
            my_ind += 1
            break
        my_ind += 1
    # correction if we miss the mark too far
    if sum(count_dict.values())%2 == 0:
        median = (sk[my_ind] + sk[my_ind-1])//2
    else:
        median = sk[my_ind]
    return median

time: 21.5 ms


In [28]:
count_d = {1:4, 2:6, 3:3, 4:8, 5:3, 10:2}

time: 7.22 ms


In [30]:
list = []
for key in count_d.keys():
    for i in range(count_d[key]):
        list.append(key)
list = sorted(list)

time: 4.65 ms


In [34]:
(list[13] + list[12])//2

3

time: 5.68 ms


In [29]:
quick_median_new2(count_d)

13.0


3

time: 6.58 ms


In [1]:
def count_frags_new(file: str) -> list:
    """
    Takes a sam file input and creates a counting dictionary of the number of reads that are paired,
    first in the pair, confidently mapped and whose pair is mapped to the same reference
    :param file: A sam input file
    :return: A dictionary of the counts of the above reads
    """
    FILTER_MAPQUAL = 10  # only consider reads that are mapped with at least this mapping quality
    count_list = []
    PRINT_EVERY = 100000
    i = 0
    # Check if the file is sam or bam and decide how to open based on that
    if file[-4:] == ".sam":
        file_to_parse = open(file, 'r')
    elif file[-4:] == ".bam":
        file_to_parse = pysam.AlignmentFile(file, 'rb')
    else:
        print("Unknown file type, must be bam or sam")
        exit(1)

    for item in file_to_parse:
        # Need to convert bam iterable objects into strings for the next part
        line = str(item)
        # Skip all comments and headers
        if line[0] == '#' or line[0] == '@':
            continue
        splt = line.strip().split('\t')
        samFlag = int(splt[1])
        myRef = splt[2]
        mapQual = int(splt[4])
        mateRef = splt[6]
        myTlen = abs(int(splt[8]))

        # if read is paired, and is first in pair, and is confidently mapped...
        if samFlag & 1 and samFlag & 64 and mapQual > FILTER_MAPQUAL:
            # and mate is mapped to same reference
            if mateRef == '=' or mateRef == myRef:
                count_list.append(myTlen)
    count_list = sorted(count_list)
    return count_list

In [ ]:
wrapped3 = wrapper(count_frags, "/home/joshfactorial/Documents/neat_data/WGS_chr20_21_22_normal.bam")
timeit.timeit(wrapped3, number=1)

In [ ]:
wrapped4 = wrapper(count_frags_new, "/home/joshfactorial/Documents/neat_data/WGS_chr20_21_22_normal.bam")
timeit.timeit(wrapped4, number=1)

In [2]:
def quick_median_new(datalist: list) -> float:
    """
    Finds the median of a counting dictionary
    :param count_dict: the counting dictionary to find the median of
    :return: integer index of the location of the median
    """
    midpoint = len(datalist)//2
    if len(datalist) % 2 == 0:
        median = (datalist[midpoint] + datalist[midpoint-1])/2
    else:
        median = datalist[midpoint]
    return median

In [ ]:
quick_median(cd)

In [3]:
def median_deviation_from_median_new(datalist: list) -> float:
    """
    calculates the deviation from the median of each element of counting dictionary,
    then returns the median of that dictionary
    :param count_dict: Counting dictionary to analyze
    :return: index of median of the deviations
    """
    my_median = quick_median_new(datalist)
    deviations = []
    for item in datalist:
        X_value = abs(item - my_median)
        deviations.append(X_value)
    return quick_median_new(sorted(deviations))

In [ ]:
median_deviation_from_median(cd)

In [9]:
def compute_probs_new(datalist: list) -> (list, list):
    """
    Computes the probabilities for fragments with at least 100 pairs supporting it and that are at least 10 median
    deviations from the median.
    :param count_dict: A dictionary of fragments with counts
    :return: A list of values that meet the criteria and a list of -their associated probabilities
    """
    FILTER_MINREADS = 100  # only consider fragment lengths that have at least this many read pairs supporting it
    FILTER_MEDDEV_M = 10  # only consider fragment lengths this many median deviations above the median
    values = []
    probabilities = []
    med = quick_median_new(datalist)
    mdm = median_deviation_from_median_new(datalist)
    
    for item in list(set(datalist)):
        if 0 < item < med + FILTER_MEDDEV_M * mdm:
            if datalist.count(item) >= FILTER_MINREADS:
                values.append(item)
                probabilities.append(datalist.count(item))
    count_sum = float(sum(probabilities))
    probabilities = [n / count_sum for n in probabilities]
    return values, probabilities

In [ ]:
sum(x == 10 for x in cd)

In [ ]:
list.count(10)

In [10]:
counting_list = count_frags_new("/home/joshfactorial/Documents/neat_data/WGS_chr20_21_22_normal.bam")
print("computing probs")
compute_probs_new(counting_list)

computing probs


([81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100],
 [0.0014269858886951008,
  0.0019132181174356536,
  0.0021669044976481157,
  0.0028539717773902015,
  0.0024734422070715077,
  0.007071507848422388,
  0.00800169124253475,
  0.007927699381639449,
  0.008033402040061307,
  0.007853707520744147,
  0.03276782411077639,
  0.0322287405528249,
  0.03289466730088262,
  0.03376142909994186,
  0.02844458538132234,
  0.23078061413244544,
  0.21153216003382486,
  0.1821996723217589,
  0.16061518947201522,
  0.005052587072564875])

Below are the two competing compute frag length programs, so that I can run side-by-side time comparisons.

In [17]:
# Count_dict method

def quick_median1(count_dict: dict) -> int:
    mid_point = sum(count_dict.values()) / 2
    my_sum = 0
    my_ind = 0
    sk = sorted(count_dict.keys())
    while my_sum < mid_point:
        my_sum += count_dict[sk[my_ind]]
        if my_sum >= mid_point:
            break
        my_ind += 1
    if sum(count_dict.values())%2 == 0:
        median = (sk[my_ind] + sk[my_ind-1])//2
    else:
        median = sk[my_ind]
    return median


def median_deviation_from_median1(count_dict: dict) -> int:
    """
    calculates the deviation from the median of each element of counting dictionary,
    then returns the median of that dictionary
    :param count_dict: Counting dictionary to analyze
    :return: index of median of the deviations
    """
    my_median = quick_median1(count_dict)
    deviations = {}
    for key in sorted(count_dict.keys()):
        X_value = abs(key - my_median)
        deviations[X_value] = count_dict[key]
    return quick_median1(deviations)


def count_frags1(file: str) -> dict:
    """
    Takes a sam file input and creates a counting dictionary of the number of reads that are paired,
    first in the pair, confidently mapped and whose pair is mapped to the same reference
    :param file: A sam input file
    :return: A dictionary of the counts of the above reads
    """
    FILTER_MAPQUAL = 10  # only consider reads that are mapped with at least this mapping quality
    count_dict = {}
    PRINT_EVERY = 100000
    # Check if the file is sam or bam and decide how to open based on that
    if file[-4:] == ".sam":
        file_to_parse = open(file, 'r')
    elif file[-4:] == ".bam":
        file_to_parse = pysam.AlignmentFile(file, 'rb')
    else:
        print("Unknown file type, must be bam or sam")
        exit(1)

    for item in file_to_parse:
        # Need to convert bam iterable objects into strings for the next part
        line = str(item)
        # Skip all comments and headers
        if line[0] == '#' or line[0] == '@':
            continue
        splt = line.strip().split('\t')
        samFlag = int(splt[1])
        myRef = splt[2]
        mapQual = int(splt[4])
        mateRef = splt[6]
        myTlen = abs(int(splt[8]))

        # if read is paired, and is first in pair, and is confidently mapped...
        if samFlag & 1 and samFlag & 64 and mapQual > FILTER_MAPQUAL:
            # and mate is mapped to same reference
            if mateRef == '=' or mateRef == myRef:
                if myTlen not in count_dict:
                    count_dict[myTlen] = 0
                count_dict[myTlen] += 1
    return count_dict


def compute_probs1(count_dict: dict) -> (list, list):
    """
    Computes the probabilities for fragments with at least 100 pairs supporting it and that are at least 10 median
    deviations from the median.
    :param count_dict: A dictionary of fragments with counts
    :return: A list of values that meet the criteria and a list of -their associated probabilities
    """
    FILTER_MINREADS = 100  # only consider fragment lengths that have at least this many read pairs supporting it
    FILTER_MEDDEV_M = 10  # only consider fragment lengths this many median deviations above the median
    values = []
    probabilities = []
    med = quick_median1(count_dict)
    mdm = median_deviation_from_median1(count_dict)

    for key in sorted(count_dict.keys()):
        if 0 < key < med + FILTER_MEDDEV_M * mdm:
            if count_dict[key] >= FILTER_MINREADS:
                values.append(key)
                probabilities.append(count_dict[key])
    count_sum = float(sum(probabilities))
    probabilities = [n / count_sum for n in probabilities]
    return values, probabilities

time: 22.8 ms


In [21]:
counting_list = count_frags1("/home/joshfactorial/Documents/neat_data/WGS_chr20_21_22_normal.bam")

time: 24min 3s


In [22]:
compute_probs1(counting_list)

{81: 13, 80: 3, 79: 8, 78: 4, 77: 2, 76: 5, 75: 6, 74: 7, 73: 9, 72: 13, 71: 3, 70: 5, 69: 9, 68: 3, 67: 9, 66: 2, 65: 8, 64: 6, 63: 7, 62: 6, 61: 7, 60: 10, 59: 10, 58: 5, 57: 12, 56: 8, 55: 5, 54: 5, 53: 12, 52: 8, 51: 12, 50: 5, 49: 11, 48: 11, 47: 7, 46: 10, 45: 10, 44: 14, 43: 8, 42: 5, 41: 10, 40: 7, 39: 13, 38: 7, 37: 10, 36: 16, 35: 6, 34: 11, 33: 9, 32: 16, 31: 14, 30: 9, 29: 25, 28: 19, 27: 20, 26: 17, 25: 23, 24: 34, 23: 55, 22: 42, 21: 37, 20: 81, 19: 135, 18: 181, 17: 205, 16: 270, 15: 234, 14: 669, 13: 757, 12: 750, 11: 760, 10: 743, 9: 3100, 8: 3049, 7: 3112, 6: 3194, 5: 2691, 4: 21833, 3: 20012, 2: 17237, 1: 12661399, 0: 478}
81 135
82 181
83 205
84 270
85 234
86 669
87 757
88 750
89 760
90 743
91 3100
92 3049
93 3112
94 3194
95 2691
96 21833
97 20012
98 17237
99 15195
100 478
101 12661399


([81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101],
 [1.0583251620178232e-05,
  1.4189396616683407e-05,
  1.607086357138176e-05,
  2.1166503240356464e-05,
  1.8344302808308934e-05,
  5.244589136221657e-05,
  5.934460352944386e-05,
  5.879584233432351e-05,
  5.9579786898781155e-05,
  5.824708113920316e-05,
  0.0002430228149818705,
  0.0002390246977031365,
  0.00024396354845921967,
  0.0002503918938877724,
  0.00021095948229555274,
  0.00171158616758038,
  0.0015688298623926426,
  0.0013512852457556457,
  0.0011912037656933942,
  3.747255018107551e-05,
  0.9925834924479484])

time: 21.4 ms


In [19]:
# Datalist method

def quick_median_new(datalist: list) -> float:
    """
    Finds the median of a counting dictionary
    :param count_dict: the counting dictionary to find the median of
    :return: integer index of the location of the median
    """
    midpoint = len(datalist)//2
    if len(datalist) % 2 == 0:
        median = (datalist[midpoint] + datalist[midpoint-1])/2
    else:
        median = datalist[midpoint]
    return median


def median_deviation_from_median_new(datalist: list) -> float:
    """
    calculates the deviation from the median of each element of counting dictionary,
    then returns the median of that dictionary
    :param count_dict: Counting dictionary to analyze
    :return: index of median of the deviations
    """
    my_median = quick_median_new(datalist)
    deviations = []
    for item in datalist:
        X_value = abs(item - my_median)
        deviations.append(X_value)
    return quick_median_new(sorted(deviations))


def count_frags_new(file: str) -> list:
    """
    Takes a sam file input and creates a counting dictionary of the number of reads that are paired,
    first in the pair, confidently mapped and whose pair is mapped to the same reference
    :param file: A sam input file
    :return: A dictionary of the counts of the above reads
    """
    FILTER_MAPQUAL = 10  # only consider reads that are mapped with at least this mapping quality
    count_list = []
    PRINT_EVERY = 100000
    i = 0
    # Check if the file is sam or bam and decide how to open based on that
    if file[-4:] == ".sam":
        file_to_parse = open(file, 'r')
    elif file[-4:] == ".bam":
        file_to_parse = pysam.AlignmentFile(file, 'rb')
    else:
        print("Unknown file type, must be bam or sam")
        exit(1)

    for item in file_to_parse:
        # Need to convert bam iterable objects into strings for the next part
        line = str(item)
        # Skip all comments and headers
        if line[0] == '#' or line[0] == '@':
            continue
        splt = line.strip().split('\t')
        samFlag = int(splt[1])
        myRef = splt[2]
        mapQual = int(splt[4])
        mateRef = splt[6]
        myTlen = abs(int(splt[8]))

        # if read is paired, and is first in pair, and is confidently mapped...
        if samFlag & 1 and samFlag & 64 and mapQual > FILTER_MAPQUAL:
            # and mate is mapped to same reference
            if mateRef == '=' or mateRef == myRef:
                count_list.append(myTlen)
    count_list = sorted(count_list)
    file_to_parse.close()
    return count_list


def compute_probs_new(datalist: list) -> (list, list):
    """
    Computes the probabilities for fragments with at least 100 pairs supporting it and that are at least 10 median
    deviations from the median.
    :param count_dict: A dictionary of fragments with counts
    :return: A list of values that meet the criteria and a list of -their associated probabilities
    """
    FILTER_MINREADS = 100  # only consider fragment lengths that have at least this many read pairs supporting it
    FILTER_MEDDEV_M = 10  # only consider fragment lengths this many median deviations above the median
    values = []
    probabilities = []
    med = quick_median_new(datalist)
    mdm = median_deviation_from_median_new(datalist)
    
    for item in list(set(datalist)):
        if 0 < item < med + FILTER_MEDDEV_M * mdm:
            data_count = datalist.count(item)
            if data_count >= FILTER_MINREADS:
                values.append(item)
                probabilities.append(data_count)
    count_sum = float(sum(probabilities))
    probabilities = [n / count_sum for n in probabilities]
    return values, probabilities

time: 22.3 ms


In [37]:
counting_list = count_frags_new("/home/joshfactorial/Documents/neat_data/WGS_chr20_21_22_normal.bam")

time: 18min 4s


In [24]:
compute_probs_new(counting_list)

([81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100],
 [0.0014269858886951008,
  0.0019132181174356536,
  0.0021669044976481157,
  0.0028539717773902015,
  0.0024734422070715077,
  0.007071507848422388,
  0.00800169124253475,
  0.007927699381639449,
  0.008033402040061307,
  0.007853707520744147,
  0.03276782411077639,
  0.0322287405528249,
  0.03289466730088262,
  0.03376142909994186,
  0.02844458538132234,
  0.23078061413244544,
  0.21153216003382486,
  0.1821996723217589,
  0.16061518947201522,
  0.005052587072564875])

time: 1min 13s
